Version Check

In [7]:
from langchain import __version__ as MainVersion
from langchain_core import __version__ as CoreVersion

if MainVersion>='1.0':
    print("Correct Version")

else:
    print("pip install -U pip")

print(f"Langchain Version:{MainVersion}")
print(f"Langchain Core Version:{CoreVersion}")

Correct Version
Langchain Version:1.2.0
Langchain Core Version:1.2.5


LOAD ENV VARIABLES

In [89]:
from dotenv import load_dotenv
import os

load_dotenv()

if os.getenv("OPENAI_API_KEY"):
    print("OPENAI API KEY FOUND")
else:
    print("OPENAI API KEY NOT FOUND")


OPENAI API KEY FOUND


Langchain Document Module

In [11]:
from langchain_core.documents import Document

DOCS=[Document(
    page_content="I love to Code",
    metadata={
        "Source":"My Life My Rules",
        "author":"Nilarghya Sarkar"
    }
),
Document(
    page_content="I love to architect things",
    metadata={
        "Source":"My Life My Rules",
        "author":"Nilarghya Sarkar"
    }
),
Document(
    page_content="I want to learn HLD and LLD",
    metadata={
        "Source":"Little wierdo",
        "author":"Aparna Nag"
    }
)
]

for index,elements in enumerate(DOCS):
    print(f"Index is {index} for the element {elements}")

Index is 0 for the element page_content='I love to Code' metadata={'Source': 'My Life My Rules', 'author': 'Nilarghya Sarkar'}
Index is 1 for the element page_content='I love to architect things' metadata={'Source': 'My Life My Rules', 'author': 'Nilarghya Sarkar'}
Index is 2 for the element page_content='I want to learn HLD and LLD' metadata={'Source': 'Little wierdo', 'author': 'Aparna Nag'}


Runnable Lambda

(It helps to include custom python functions in the Langchain chain)

In [ ]:
from langchain_core.runnables import RunnableLambda                                 
'''#RunnableLambda is used for Wrap a Python function so it becomes a LangChain runnable.
    input ──▶ RunnableLambda(fn) ──▶ transformed output'''

def set_to_upper(param:str)->str:
    """Convert the param to UpperCase"""
    print(f"{param.upper()}")

def set_to_lower(param:str)->str:
    """Convert the param to LowerCase"""
    print(f"{param.lower()}")


uppercase_runnable=RunnableLambda(set_to_upper)
lowercase_runnable=RunnableLambda(set_to_lower)

chain=lowercase_runnable

result=chain.invoke("hello")

hello


Chat Invoking

In [19]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=0)

response=llm.invoke("Who is the President of Venezuala ?")

print(response.content)
print('\n')
print(response.response_metadata)


As of September 2021, the President of Venezuela is Nicolás Maduro.


{'token_usage': {'completion_tokens': 16, 'prompt_tokens': 15, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cvf5FNkQwqlNlMn8a5mWheuCzoH6i', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}


Prompting

In [90]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langfuse.langchain import CallbackHandler

langfuse_handler=CallbackHandler()
llm=ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)
prompt=ChatPromptTemplate.from_template(
    "Explain the {Topic} in 3 words"
)
print(prompt.messages[0].prompt.template)
chain=prompt|llm
response=chain.invoke("Science",config={'callbacks':[langfuse_handler]})
print(response.content)

Explain the {Topic} in 3 words
Observation, experimentation, explanation.


Stroutput Parser
(To get only the text from the LLM response)

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm=ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0
)

prompt=ChatPromptTemplate.from_template(
    "Explain the {topic} to a human of age {age}"
)
chain=prompt|llm|StrOutputParser()

response=chain.invoke({
    "topic":"Cow",
    "age":32
})

print(response)

A cow is a large domesticated mammal that is commonly raised for its meat, milk, and other products. They have a distinctive appearance with a large body, four legs, and a long tail. Cows are herbivores, meaning they primarily eat grass and other plant-based foods.

Cows are known for their gentle nature and are often found grazing in fields or pastures. They are social animals that live in herds and have a complex social structure. Cows are also known for their ability to produce milk, which is a valuable source of nutrition for humans.

Overall, cows play an important role in agriculture and food production, providing us with meat, dairy products, and other resources. They are a common sight in rural areas and are an important part of many people's lives.


Batch Processing

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm=ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0
)
prompts=PromptTemplate.from_template(
    "Give details about the {topic} in 1-2 lines"
)
chain=prompts|llm|StrOutputParser()
batch=[
    {"topic":"RAG"},
    {"topic":"LCEL"}
]

results=chain.batch(batch)
for index,(b,r) in enumerate(zip(batch,results)):
    print(f"{index}. {b['topic']}")
    print(r)

0. RAG
The RAG (Red, Amber, Green) rating system is a project management tool used to visually communicate the status of tasks or projects. Red indicates critical issues, amber signifies potential problems, and green represents tasks on track.
1. LCEL
The LCEL (Low Carbon Economy Lab) is a collaborative initiative between the Government of Canada and the provinces of Alberta, British Columbia, Ontario, and Quebec to accelerate the development and adoption of clean technologies and reduce greenhouse gas emissions.


Document Loader

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

pdf=Path("PDFs/BLR CCU.pdf")
if not pdf.exists():
    print("Check PDF location")

loader=PyPDFLoader(pdf)

result=loader.load()
print(result[0].page_content[:104])


E­Ticket
Booking ID ­ EMT156142859
Booking on ­Mon 01 Sep 2025 09:44 AM
Onward Flight
Bengaluru
 Kolkata


Multiple Pdfs Loading

In [63]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

pdf_directory=Path('./PDFs')
all_docs=[]


all_docs=list(pdf_directory.glob('*.pdf'))
for index,pdf in enumerate(all_docs):
    loader=PyPDFLoader(pdf)
    result=loader.load()
    print(f"{index: }")
    print(result[0].metadata)


    



 0
{'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'source': 'PDFs/visit-prescription.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}
 1
{'producer': 'Select.Pdf Html To Pdf for .NET Community Edition v19.2.0', 'creator': 'PyPDF', 'creationdate': '', 'source': 'PDFs/BLR CCU.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}


Text Splitter

In [75]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path
text_splits=RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
)
pdf=Path('PDFs/BLR CCU.pdf')
if pdf.exists():
    loader=PyPDFLoader(pdf)
    loaded_docs=loader.load()
    text=loaded_docs[0].page_content





chunks=text_splits.split_documents(loaded_docs)
for index,chunk in enumerate(chunks[:3]):
    print(f"Chunk {index+1}")
    print(f"Chunked Text {chunk}")

Chunk 1
Chunked Text page_content='E­Ticket
Booking ID ­ EMT156142859
Booking on ­Mon 01 Sep 2025 09:44 AM
Onward Flight
Bengaluru' metadata={'producer': 'Select.Pdf Html To Pdf for .NET Community Edition v19.2.0', 'creator': 'PyPDF', 'creationdate': '', 'source': 'PDFs/BLR CCU.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}
Chunk 2
Chunked Text page_content='Bengaluru
 Kolkata
Thursday, September 25, 2025 PNR: D8M7TA
IndiGo
6E­ 6277
Thu 25 Sep 2025' metadata={'producer': 'Select.Pdf Html To Pdf for .NET Community Edition v19.2.0', 'creator': 'PyPDF', 'creationdate': '', 'source': 'PDFs/BLR CCU.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}
Chunk 3
Chunked Text page_content='BLR 14:45
Kempegowda International Airport, Terminal 1
02h 30m
Thu­25Sep2025
PAT17:15' metadata={'producer': 'Select.Pdf Html To Pdf for .NET Community Edition v19.2.0', 'creator': 'PyPDF', 'creationdate': '', 'source': 'PDFs/BLR CCU.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}


Embedding

In [76]:
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings(
    model='text-embedding-3-small'
)

sample_text='How are you Boss'
embedded_text=embeddings.embed_query(sample_text)

print(f"Dimension of the Embedding {len(embedded_text)}")
print(f"Embeddings are {embedded_text[:10]}")

Dimension of the Embedding 1536
Embeddings are [0.04930621385574341, -0.05263548344373703, -0.046142030507326126, 0.01465153880417347, 0.0025554210878908634, -0.03808024525642395, -0.0002536504762247205, -0.01455523818731308, 0.0033567845821380615, -0.014857898466289043]


Vector Store

In [78]:
from langchain_community.vectorstores import FAISS

from langchain_openai import OpenAIEmbeddings

sample_query="Ram is 12 years ols and studies in rkmv he a good coder"
embedding=OpenAIEmbeddings(
    model='text-embedding-3-small'
)


vectorstore=FAISS.from_texts(
    texts=sample_query,
    embedding=embedding
)

vectorstore_path='./vector_store'

vectorstore.save_local(vectorstore_path)


Vector Store as Retriever

In [ ]:
retriever=vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k':5}
)

search_query="what is Ram's age"
retrieved_text=retriever.invoke(search_query)

print(retrieved_text)

[Document(id='84c6c050-ed86-4e54-b23c-f599e786a3e8', metadata={}, page_content='r'), Document(id='93db4352-215b-4381-98f0-5bb305d7122b', metadata={}, page_content='r'), Document(id='4e7beddf-8b48-480b-adeb-4a0d3cf99c88', metadata={}, page_content='r'), Document(id='27d7717f-7bd3-48d8-9423-083f0b1e2897', metadata={}, page_content='R'), Document(id='93228688-a786-4d1f-a2a4-7bcce4a14921', metadata={}, page_content='1')]
